<a href="https://colab.research.google.com/github/Digital-AI-Finance/Digital-Finance-Introduction/blob/main/day_04/notebooks/NB08_Smart_Contracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NB08: Smart Contracts

**Topic:** 4.1 - Programmable Finance

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand Smart Contracts**: Explain what smart contracts are and how they enable programmable finance
2. **Read Solidity Code**: Analyze and understand simple Solidity smart contract code
3. **Simulate Contract Execution**: Implement Python simulations of token and escrow contracts
4. **Understand State Management**: Track how contract state transitions occur through function calls
5. **Estimate Gas Costs**: Understand how different operations consume computational resources
6. **Recognize Vulnerabilities**: Identify common security issues like reentrancy and overflow attacks

## Section 1: Setup

This notebook uses pure Python to simulate smart contract behavior. No blockchain connection or web3.py required - we're building educational simulations from scratch!

In [ ]:
# Standard library imports only - no external dependencies needed!
import hashlib
import time
from datetime import datetime
from typing import Dict, List, Any, Optional, Callable
from dataclasses import dataclass, field
from enum import Enum
import json

print("Libraries imported successfully!")
print("\nThis notebook simulates smart contracts in pure Python.")
print("No blockchain connection needed - we're learning the concepts!")

## Section 2: What are Smart Contracts?

### The Traditional Contract Problem

Traditional contracts require:
- **Trusted intermediaries**: Lawyers, banks, escrow services
- **Manual enforcement**: Courts, arbitration, legal action
- **Subjective interpretation**: "Reasonable effort", "good faith"
- **Time delays**: Days to weeks for settlement

### The Smart Contract Solution

A **smart contract** is:
- **Self-executing code** stored on a blockchain
- **Automatically enforced** by the network
- **Deterministic**: Same inputs always produce same outputs
- **Immutable**: Once deployed, code cannot be changed
- **Trustless**: No intermediaries needed

### Nick Szabo's Vision (1994)

> "A smart contract is a computerized transaction protocol that executes the terms of a contract."
> 
> The vending machine is the simplest example:
> - Insert coins (input)
> - Select product (function call)
> - Receive product (output)
> - No trust required - the machine enforces the rules

In [ ]:
# Let's start with the simplest "smart contract" - a vending machine

class VendingMachine:
    """
    The simplest example of a smart contract:
    - Has state (inventory, balance)
    - Has rules (prices, stock limits)
    - Executes automatically based on inputs
    - No trust required between buyer and seller
    """
    
    def __init__(self):
        # Contract state
        self.inventory = {
            "cola": {"price": 150, "stock": 10},  # Price in cents
            "chips": {"price": 200, "stock": 5},
            "candy": {"price": 100, "stock": 15}
        }
        self.balance = 0  # Coins inserted
        self.owner_revenue = 0
    
    def insert_coins(self, amount: int) -> str:
        """Add coins to the machine."""
        if amount <= 0:
            return "ERROR: Invalid amount"
        self.balance += amount
        return f"Balance: {self.balance} cents"
    
    def select_product(self, product: str) -> str:
        """Attempt to purchase a product."""
        # Check if product exists
        if product not in self.inventory:
            return "ERROR: Product not found"
        
        item = self.inventory[product]
        
        # Check stock
        if item["stock"] <= 0:
            return "ERROR: Out of stock"
        
        # Check payment
        if self.balance < item["price"]:
            return f"ERROR: Insufficient funds. Need {item['price']} cents, have {self.balance}"
        
        # Execute transaction
        self.balance -= item["price"]
        item["stock"] -= 1
        self.owner_revenue += item["price"]
        
        change = self.balance
        self.balance = 0
        
        return f"SUCCESS: Dispensing {product}. Change: {change} cents"
    
    def get_state(self) -> dict:
        """View current machine state."""
        return {
            "inventory": self.inventory.copy(),
            "balance": self.balance,
            "owner_revenue": self.owner_revenue
        }

# Demonstrate the vending machine
print("=" * 70)
print("VENDING MACHINE - Simplest Smart Contract Example")
print("=" * 70)

machine = VendingMachine()

print("\n1. Initial state:")
print(f"   Inventory: {machine.inventory}")

print("\n2. Insert 200 cents:")
print(f"   {machine.insert_coins(200)}")

print("\n3. Try to buy cola (150 cents):")
print(f"   {machine.select_product('cola')}")

print("\n4. Insert only 100 cents and try chips (200 cents):")
print(f"   {machine.insert_coins(100)}")
print(f"   {machine.select_product('chips')}")

print("\n5. Final state:")
state = machine.get_state()
print(f"   Cola stock: {state['inventory']['cola']['stock']}")
print(f"   Owner revenue: {state['owner_revenue']} cents")

print("\n" + "=" * 70)
print("KEY INSIGHT: The machine enforces rules automatically.")
print("No trust needed - the code IS the contract!")
print("=" * 70)

### Smart Contracts vs Traditional Contracts

| Aspect | Traditional Contract | Smart Contract |
|--------|---------------------|----------------|
| Enforcement | Courts, lawyers | Automatic by code |
| Trust | In counterparties | In code/protocol |
| Execution | Manual | Automatic |
| Cost | High (legal fees) | Low (gas fees) |
| Speed | Days to months | Seconds to minutes |
| Ambiguity | Possible | Impossible (code is precise) |
| Modification | Negotiable | Immutable |
| Transparency | Private | Public (on-chain) |

## Section 3: Anatomy of a Smart Contract

Smart contracts on Ethereum are written in **Solidity**, a high-level language that compiles to EVM bytecode.

Let's examine the key components of a Solidity contract:

In [ ]:
# Solidity code example - displayed as a string for educational purposes
# This is a simple token contract

SIMPLE_TOKEN_SOLIDITY = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

/**
 * @title SimpleToken
 * @dev A basic ERC20-like token for educational purposes
 */
contract SimpleToken {
    // ============================================
    // STATE VARIABLES (stored on blockchain)
    // ============================================
    
    string public name = "SimpleToken";          // Token name
    string public symbol = "STKN";               // Token symbol
    uint8 public decimals = 18;                  // Decimal places
    uint256 public totalSupply;                  // Total tokens in existence
    
    // Mapping: address => balance
    mapping(address => uint256) public balanceOf;
    
    // Mapping: owner => spender => allowance
    mapping(address => mapping(address => uint256)) public allowance;
    
    // Contract owner
    address public owner;
    
    // ============================================
    // EVENTS (logs emitted to blockchain)
    // ============================================
    
    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);
    event Mint(address indexed to, uint256 amount);
    
    // ============================================
    // CONSTRUCTOR (runs once at deployment)
    // ============================================
    
    constructor(uint256 initialSupply) {
        owner = msg.sender;                      // Deployer becomes owner
        totalSupply = initialSupply * 10**decimals;
        balanceOf[msg.sender] = totalSupply;     // Give all tokens to owner
        emit Transfer(address(0), msg.sender, totalSupply);
    }
    
    // ============================================
    // MODIFIERS (reusable access control)
    // ============================================
    
    modifier onlyOwner() {
        require(msg.sender == owner, "Not the owner");
        _;  // Continue with function execution
    }
    
    // ============================================
    // FUNCTIONS (callable by users)
    // ============================================
    
    /**
     * @dev Transfer tokens to another address
     * @param to Recipient address
     * @param amount Number of tokens to transfer
     */
    function transfer(address to, uint256 amount) public returns (bool) {
        require(to != address(0), "Invalid recipient");
        require(balanceOf[msg.sender] >= amount, "Insufficient balance");
        
        balanceOf[msg.sender] -= amount;         // Deduct from sender
        balanceOf[to] += amount;                  // Add to recipient
        
        emit Transfer(msg.sender, to, amount);   // Log the transfer
        return true;
    }
    
    /**
     * @dev Approve another address to spend tokens on your behalf
     */
    function approve(address spender, uint256 amount) public returns (bool) {
        allowance[msg.sender][spender] = amount;
        emit Approval(msg.sender, spender, amount);
        return true;
    }
    
    /**
     * @dev Transfer tokens from one address to another (requires approval)
     */
    function transferFrom(address from, address to, uint256 amount) public returns (bool) {
        require(allowance[from][msg.sender] >= amount, "Allowance exceeded");
        require(balanceOf[from] >= amount, "Insufficient balance");
        
        allowance[from][msg.sender] -= amount;
        balanceOf[from] -= amount;
        balanceOf[to] += amount;
        
        emit Transfer(from, to, amount);
        return true;
    }
    
    /**
     * @dev Mint new tokens (only owner can call)
     */
    function mint(address to, uint256 amount) public onlyOwner {
        totalSupply += amount;
        balanceOf[to] += amount;
        emit Mint(to, amount);
        emit Transfer(address(0), to, amount);
    }
}
'''

print("SOLIDITY SMART CONTRACT EXAMPLE")
print("=" * 70)
print(SIMPLE_TOKEN_SOLIDITY)
print("=" * 70)

In [ ]:
# Let's break down the key components

print("\n" + "=" * 70)
print("SMART CONTRACT ANATOMY")
print("=" * 70)

components = [
    ("1. PRAGMA", 
     "pragma solidity ^0.8.0;",
     "Specifies compiler version. ^0.8.0 means any version >= 0.8.0 and < 0.9.0"),
    
    ("2. STATE VARIABLES",
     "mapping(address => uint256) public balanceOf;",
     "Permanent storage on blockchain. Each read/write costs gas."),
    
    ("3. EVENTS",
     "event Transfer(address indexed from, address indexed to, uint256 value);",
     "Logs stored on blockchain. Cheaper than storage, used for notifications."),
    
    ("4. CONSTRUCTOR",
     "constructor(uint256 initialSupply) { ... }",
     "Runs ONCE when contract is deployed. Sets up initial state."),
    
    ("5. MODIFIERS",
     "modifier onlyOwner() { require(msg.sender == owner); _; }",
     "Reusable access control. The _ is where the function body executes."),
    
    ("6. FUNCTIONS",
     "function transfer(address to, uint256 amount) public returns (bool)",
     "Callable code. 'public' = anyone can call. Can be view/pure (read-only)."),
    
    ("7. REQUIRE",
     "require(balanceOf[msg.sender] >= amount, 'Insufficient balance');",
     "Validation that reverts the transaction if condition is false."),
    
    ("8. MSG.SENDER",
     "msg.sender",
     "Global variable containing the address that called the function."),
]

for name, code, description in components:
    print(f"\n{name}")
    print(f"   Code: {code}")
    print(f"   Meaning: {description}")

print("\n" + "=" * 70)

## Section 4: Python Simulation - Token Contract

Let's implement the Solidity token contract in Python to understand how it works:

In [ ]:
@dataclass
class Event:
    """Represents a blockchain event/log."""
    name: str
    data: Dict[str, Any]
    timestamp: float = field(default_factory=time.time)
    
    def __str__(self):
        return f"Event({self.name}): {self.data}"


class SimulatedTokenContract:
    """
    Python simulation of an ERC20-like token contract.
    Mirrors the Solidity contract structure for educational purposes.
    """
    
    def __init__(self, deployer: str, name: str, symbol: str, initial_supply: int):
        """
        Constructor - runs once at deployment.
        
        Args:
            deployer: Address that deploys the contract (becomes owner)
            name: Token name
            symbol: Token symbol
            initial_supply: Initial token supply (before decimals)
        """
        # State variables
        self.name = name
        self.symbol = symbol
        self.decimals = 18
        self.owner = deployer
        
        # Calculate total supply with decimals
        self.total_supply = initial_supply * (10 ** self.decimals)
        
        # Mappings (dictionaries in Python)
        self.balance_of: Dict[str, int] = {}
        self.allowance: Dict[str, Dict[str, int]] = {}
        
        # Give all tokens to deployer
        self.balance_of[deployer] = self.total_supply
        
        # Event log
        self.events: List[Event] = []
        
        # Emit initial transfer event
        self._emit_event("Transfer", {
            "from": "0x0000000000000000000000000000000000000000",
            "to": deployer,
            "value": self.total_supply
        })
        
        print(f"Contract deployed by {deployer[:10]}...")
        print(f"Token: {name} ({symbol})")
        print(f"Total supply: {self._format_tokens(self.total_supply)}")
    
    def _emit_event(self, name: str, data: Dict[str, Any]):
        """Emit an event (log to blockchain)."""
        event = Event(name, data)
        self.events.append(event)
    
    def _format_tokens(self, amount: int) -> str:
        """Format token amount with decimals."""
        return f"{amount / (10 ** self.decimals):,.4f} {self.symbol}"
    
    def _require(self, condition: bool, message: str):
        """Solidity-style require - reverts if condition is false."""
        if not condition:
            raise ValueError(f"REVERT: {message}")
    
    def transfer(self, sender: str, to: str, amount: int) -> bool:
        """
        Transfer tokens from sender to recipient.
        
        Args:
            sender: msg.sender - who is calling this function
            to: Recipient address
            amount: Amount of tokens (with decimals)
        """
        # Validation (require statements)
        self._require(to != "0x0000000000000000000000000000000000000000", "Invalid recipient")
        self._require(self.balance_of.get(sender, 0) >= amount, "Insufficient balance")
        
        # State changes
        self.balance_of[sender] = self.balance_of.get(sender, 0) - amount
        self.balance_of[to] = self.balance_of.get(to, 0) + amount
        
        # Emit event
        self._emit_event("Transfer", {"from": sender, "to": to, "value": amount})
        
        print(f"Transfer: {sender[:10]}... -> {to[:10]}... : {self._format_tokens(amount)}")
        return True
    
    def approve(self, owner: str, spender: str, amount: int) -> bool:
        """
        Approve spender to spend tokens on behalf of owner.
        
        Args:
            owner: msg.sender - who is giving approval
            spender: Address being approved
            amount: Maximum amount spender can transfer
        """
        if owner not in self.allowance:
            self.allowance[owner] = {}
        self.allowance[owner][spender] = amount
        
        self._emit_event("Approval", {"owner": owner, "spender": spender, "value": amount})
        
        print(f"Approval: {owner[:10]}... approved {spender[:10]}... for {self._format_tokens(amount)}")
        return True
    
    def transfer_from(self, caller: str, from_addr: str, to: str, amount: int) -> bool:
        """
        Transfer tokens from one address to another (requires prior approval).
        
        Args:
            caller: msg.sender - who is calling this function
            from_addr: Address to transfer from
            to: Recipient address
            amount: Amount to transfer
        """
        # Check allowance
        current_allowance = self.allowance.get(from_addr, {}).get(caller, 0)
        self._require(current_allowance >= amount, "Allowance exceeded")
        self._require(self.balance_of.get(from_addr, 0) >= amount, "Insufficient balance")
        
        # Update allowance
        self.allowance[from_addr][caller] -= amount
        
        # Transfer
        self.balance_of[from_addr] -= amount
        self.balance_of[to] = self.balance_of.get(to, 0) + amount
        
        self._emit_event("Transfer", {"from": from_addr, "to": to, "value": amount})
        
        print(f"TransferFrom: {from_addr[:10]}... -> {to[:10]}... : {self._format_tokens(amount)} (by {caller[:10]}...)")
        return True
    
    def mint(self, caller: str, to: str, amount: int):
        """
        Mint new tokens (only owner can call).
        
        Args:
            caller: msg.sender
            to: Address to receive new tokens
            amount: Amount to mint
        """
        # Only owner modifier
        self._require(caller == self.owner, "Not the owner")
        
        self.total_supply += amount
        self.balance_of[to] = self.balance_of.get(to, 0) + amount
        
        self._emit_event("Mint", {"to": to, "amount": amount})
        self._emit_event("Transfer", {
            "from": "0x0000000000000000000000000000000000000000",
            "to": to,
            "value": amount
        })
        
        print(f"Mint: Created {self._format_tokens(amount)} for {to[:10]}...")
    
    def get_balance(self, address: str) -> int:
        """View function - read balance without modifying state."""
        return self.balance_of.get(address, 0)
    
    def get_state_summary(self) -> dict:
        """Get a summary of the contract state."""
        return {
            "name": self.name,
            "symbol": self.symbol,
            "total_supply": self._format_tokens(self.total_supply),
            "owner": self.owner[:10] + "...",
            "holders": len([a for a, b in self.balance_of.items() if b > 0]),
            "events_count": len(self.events)
        }

In [ ]:
# Demonstrate the token contract

print("=" * 70)
print("SIMULATED TOKEN CONTRACT DEMONSTRATION")
print("=" * 70)

# Create addresses (in real life, these are derived from private keys)
ALICE = "0xAlice1234567890abcdef1234567890abcdef1234"
BOB = "0xBob123456789abcdef01234567890abcdef01234"
CHARLIE = "0xCharlie7890abcdef01234567890abcdef012345"
EXCHANGE = "0xExchange890abcdef01234567890abcdef0123456"

# Deploy contract (Alice is the deployer)
print("\n--- DEPLOYMENT ---")
token = SimulatedTokenContract(
    deployer=ALICE,
    name="Digital Finance Token",
    symbol="DFT",
    initial_supply=1_000_000  # 1 million tokens
)

# Check initial balances
print("\n--- INITIAL BALANCES ---")
print(f"Alice: {token._format_tokens(token.get_balance(ALICE))}")
print(f"Bob: {token._format_tokens(token.get_balance(BOB))}")

# Transfer tokens
print("\n--- TRANSFERS ---")
token.transfer(ALICE, BOB, 10_000 * 10**18)  # 10,000 tokens
token.transfer(ALICE, CHARLIE, 5_000 * 10**18)  # 5,000 tokens

# Check balances after transfer
print("\n--- BALANCES AFTER TRANSFER ---")
print(f"Alice: {token._format_tokens(token.get_balance(ALICE))}")
print(f"Bob: {token._format_tokens(token.get_balance(BOB))}")
print(f"Charlie: {token._format_tokens(token.get_balance(CHARLIE))}")

# Demonstrate approve and transferFrom
print("\n--- APPROVE & TRANSFER_FROM (DeFi pattern) ---")
print("Bob approves Exchange to spend his tokens...")
token.approve(BOB, EXCHANGE, 5_000 * 10**18)  # Approve 5,000

print("\nExchange transfers Bob's tokens to Charlie...")
token.transfer_from(EXCHANGE, BOB, CHARLIE, 3_000 * 10**18)

# Try to transfer more than balance (should fail)
print("\n--- ERROR HANDLING ---")
print("Bob tries to send more tokens than he has...")
try:
    token.transfer(BOB, CHARLIE, 100_000 * 10**18)
except ValueError as e:
    print(f"   {e}")

# Mint new tokens (only owner)
print("\n--- MINTING (only owner) ---")
print("Alice (owner) mints new tokens...")
token.mint(ALICE, BOB, 1_000 * 10**18)

print("\nBob tries to mint (not owner)...")
try:
    token.mint(BOB, BOB, 1_000_000 * 10**18)
except ValueError as e:
    print(f"   {e}")

# Final state
print("\n--- FINAL STATE ---")
print(json.dumps(token.get_state_summary(), indent=2))

print("\n--- EVENT LOG ---")
for event in token.events[-5:]:
    print(f"   {event}")

print("\n" + "=" * 70)

## Section 5: Python Simulation - Escrow Contract

An escrow contract is a classic smart contract use case. It holds funds until conditions are met, then releases them to the appropriate party.

**Use case**: Alice wants to buy a digital item from Bob. Neither trusts the other. The escrow contract:
1. Holds Alice's payment
2. Waits for both parties to confirm delivery
3. Releases funds to Bob (or refunds Alice if there's a dispute)

In [ ]:
# First, let's see the Solidity version

ESCROW_SOLIDITY = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

/**
 * @title Escrow
 * @dev Simple escrow contract for peer-to-peer trades
 */
contract Escrow {
    enum State { AWAITING_PAYMENT, AWAITING_DELIVERY, COMPLETE, REFUNDED }
    
    address public buyer;
    address public seller;
    address public arbiter;  // Third party for disputes
    
    uint256 public amount;
    State public state;
    
    bool public buyerConfirmed;
    bool public sellerConfirmed;
    
    event PaymentDeposited(address buyer, uint256 amount);
    event DeliveryConfirmed(address by);
    event FundsReleased(address to, uint256 amount);
    event FundsRefunded(address to, uint256 amount);
    
    constructor(address _seller, address _arbiter) {
        buyer = msg.sender;
        seller = _seller;
        arbiter = _arbiter;
        state = State.AWAITING_PAYMENT;
    }
    
    function deposit() external payable {
        require(msg.sender == buyer, "Only buyer can deposit");
        require(state == State.AWAITING_PAYMENT, "Already funded");
        require(msg.value > 0, "Must send ETH");
        
        amount = msg.value;
        state = State.AWAITING_DELIVERY;
        emit PaymentDeposited(buyer, amount);
    }
    
    function confirmDelivery() external {
        require(state == State.AWAITING_DELIVERY, "Not awaiting delivery");
        
        if (msg.sender == buyer) {
            buyerConfirmed = true;
        } else if (msg.sender == seller) {
            sellerConfirmed = true;
        } else {
            revert("Not authorized");
        }
        
        emit DeliveryConfirmed(msg.sender);
        
        // If both confirmed, release funds
        if (buyerConfirmed && sellerConfirmed) {
            _releaseFunds();
        }
    }
    
    function refund() external {
        require(msg.sender == arbiter, "Only arbiter can refund");
        require(state == State.AWAITING_DELIVERY, "Cannot refund");
        
        state = State.REFUNDED;
        payable(buyer).transfer(amount);
        emit FundsRefunded(buyer, amount);
    }
    
    function _releaseFunds() private {
        state = State.COMPLETE;
        payable(seller).transfer(amount);
        emit FundsReleased(seller, amount);
    }
}
'''

print("ESCROW CONTRACT (Solidity)")
print("=" * 70)
print(ESCROW_SOLIDITY[:2000] + "...")
print("=" * 70)

In [ ]:
# Now let's implement it in Python

class EscrowState(Enum):
    AWAITING_PAYMENT = "AWAITING_PAYMENT"
    AWAITING_DELIVERY = "AWAITING_DELIVERY"
    COMPLETE = "COMPLETE"
    REFUNDED = "REFUNDED"
    DISPUTED = "DISPUTED"


class SimulatedEscrowContract:
    """
    Python simulation of an escrow smart contract.
    Demonstrates trustless peer-to-peer transactions.
    """
    
    def __init__(self, buyer: str, seller: str, arbiter: str, description: str):
        """
        Create new escrow contract.
        
        Args:
            buyer: Address of the buyer (deployer)
            seller: Address of the seller
            arbiter: Address of the arbiter (dispute resolver)
            description: What is being bought/sold
        """
        self.buyer = buyer
        self.seller = seller
        self.arbiter = arbiter
        self.description = description
        
        self.amount = 0
        self.state = EscrowState.AWAITING_PAYMENT
        
        self.buyer_confirmed = False
        self.seller_confirmed = False
        
        self.events: List[Event] = []
        self.state_history: List[tuple] = [(time.time(), self.state, "Contract created")]
        
        self._emit_event("ContractCreated", {
            "buyer": buyer,
            "seller": seller,
            "arbiter": arbiter,
            "description": description
        })
        
        print(f"Escrow contract created")
        print(f"  Buyer: {buyer[:16]}...")
        print(f"  Seller: {seller[:16]}...")
        print(f"  Arbiter: {arbiter[:16]}...")
        print(f"  Item: {description}")
    
    def _emit_event(self, name: str, data: Dict[str, Any]):
        event = Event(name, data)
        self.events.append(event)
    
    def _require(self, condition: bool, message: str):
        if not condition:
            raise ValueError(f"REVERT: {message}")
    
    def _change_state(self, new_state: EscrowState, reason: str):
        old_state = self.state
        self.state = new_state
        self.state_history.append((time.time(), new_state, reason))
        print(f"  State: {old_state.value} -> {new_state.value}")
    
    def deposit(self, caller: str, amount: int) -> bool:
        """
        Buyer deposits payment into escrow.
        
        Args:
            caller: msg.sender
            amount: Amount in Wei to deposit
        """
        self._require(caller == self.buyer, "Only buyer can deposit")
        self._require(self.state == EscrowState.AWAITING_PAYMENT, "Already funded")
        self._require(amount > 0, "Must send funds")
        
        self.amount = amount
        self._change_state(EscrowState.AWAITING_DELIVERY, "Payment deposited")
        
        self._emit_event("PaymentDeposited", {"buyer": caller, "amount": amount})
        
        print(f"  Amount: {amount / 10**18:.4f} ETH")
        return True
    
    def confirm_delivery(self, caller: str) -> bool:
        """
        Buyer or seller confirms delivery was made.
        Both must confirm for funds to be released.
        
        Args:
            caller: msg.sender
        """
        self._require(self.state == EscrowState.AWAITING_DELIVERY, "Not awaiting delivery")
        
        if caller == self.buyer:
            self._require(not self.buyer_confirmed, "Already confirmed")
            self.buyer_confirmed = True
            print(f"  Buyer confirmed delivery")
        elif caller == self.seller:
            self._require(not self.seller_confirmed, "Already confirmed")
            self.seller_confirmed = True
            print(f"  Seller confirmed delivery")
        else:
            self._require(False, "Not authorized")
        
        self._emit_event("DeliveryConfirmed", {"by": caller})
        
        # Check if both confirmed
        if self.buyer_confirmed and self.seller_confirmed:
            return self._release_funds()
        
        return True
    
    def _release_funds(self) -> bool:
        """Internal function to release funds to seller."""
        self._change_state(EscrowState.COMPLETE, "Both parties confirmed")
        
        self._emit_event("FundsReleased", {
            "to": self.seller,
            "amount": self.amount
        })
        
        print(f"  Funds released to seller: {self.amount / 10**18:.4f} ETH")
        return True
    
    def refund(self, caller: str) -> bool:
        """
        Arbiter refunds buyer (dispute resolution).
        
        Args:
            caller: msg.sender (must be arbiter)
        """
        self._require(caller == self.arbiter, "Only arbiter can refund")
        self._require(self.state == EscrowState.AWAITING_DELIVERY, "Cannot refund")
        
        self._change_state(EscrowState.REFUNDED, "Arbiter initiated refund")
        
        self._emit_event("FundsRefunded", {
            "to": self.buyer,
            "amount": self.amount
        })
        
        print(f"  Funds refunded to buyer: {self.amount / 10**18:.4f} ETH")
        return True
    
    def get_state(self) -> dict:
        """Get current contract state."""
        return {
            "state": self.state.value,
            "amount": f"{self.amount / 10**18:.4f} ETH",
            "buyer_confirmed": self.buyer_confirmed,
            "seller_confirmed": self.seller_confirmed,
            "description": self.description
        }

In [ ]:
# Demonstrate the escrow contract - Happy path

print("=" * 70)
print("ESCROW CONTRACT - SUCCESSFUL TRANSACTION")
print("=" * 70)

# Addresses
ALICE = "0xAlice_Buyer_1234567890abcdef12345678"
BOB = "0xBob_Seller_1234567890abcdef123456789"
ARBITER = "0xArbiter_TrustedThirdParty_abcdef1234"

print("\n--- SCENARIO: Alice buys digital art from Bob ---")
print("\n1. CONTRACT CREATION")
escrow = SimulatedEscrowContract(
    buyer=ALICE,
    seller=BOB,
    arbiter=ARBITER,
    description="CryptoPunk #1234 NFT"
)

print("\n2. BUYER DEPOSITS PAYMENT")
escrow.deposit(ALICE, int(2.5 * 10**18))  # 2.5 ETH

print("\n3. SELLER DELIVERS ITEM (off-chain)")
print("   [Bob transfers NFT to Alice's wallet]")

print("\n4. SELLER CONFIRMS DELIVERY")
escrow.confirm_delivery(BOB)

print("\n5. BUYER CONFIRMS RECEIPT")
escrow.confirm_delivery(ALICE)

print("\n--- FINAL STATE ---")
print(json.dumps(escrow.get_state(), indent=2))

print("\n" + "=" * 70)

In [ ]:
# Demonstrate dispute resolution

print("=" * 70)
print("ESCROW CONTRACT - DISPUTE RESOLUTION")
print("=" * 70)

print("\n--- SCENARIO: Seller never delivers, arbiter refunds ---")

print("\n1. CONTRACT CREATION")
escrow2 = SimulatedEscrowContract(
    buyer=ALICE,
    seller=BOB,
    arbiter=ARBITER,
    description="Rare Pokemon Card"
)

print("\n2. BUYER DEPOSITS PAYMENT")
escrow2.deposit(ALICE, int(0.5 * 10**18))  # 0.5 ETH

print("\n3. [Time passes... seller doesn't deliver]")
print("   Alice files a dispute with the arbiter")

print("\n4. ARBITER INVESTIGATES AND ISSUES REFUND")
escrow2.refund(ARBITER)

print("\n--- FINAL STATE ---")
print(json.dumps(escrow2.get_state(), indent=2))

# Try unauthorized refund
print("\n5. SELLER TRIES TO CALL REFUND (should fail)")
escrow3 = SimulatedEscrowContract(ALICE, BOB, ARBITER, "Test item")
escrow3.deposit(ALICE, int(1 * 10**18))
try:
    escrow3.refund(BOB)  # Bob is not the arbiter
except ValueError as e:
    print(f"   {e}")

print("\n" + "=" * 70)

## Section 6: Contract State Transitions Visualization

Smart contracts are essentially **state machines**. Each function call can transition the contract from one state to another, following predefined rules.

In [ ]:
def visualize_escrow_state_machine():
    """
    Display the escrow contract state machine.
    """
    print("\n" + "=" * 70)
    print("ESCROW CONTRACT STATE MACHINE")
    print("=" * 70)
    
    state_diagram = '''
                    +------------------+
                    | AWAITING_PAYMENT |
                    +------------------+
                            |
                            | deposit()
                            | [buyer sends ETH]
                            v
                    +-------------------+
                    | AWAITING_DELIVERY |<---------------+
                    +-------------------+                |
                        |           |                    |
            refund()    |           | confirmDelivery()  | (both not yet
            [by arbiter]|           | [by buyer/seller]  |  confirmed)
                        v           v                    |
                +----------+    +--------------+         |
                | REFUNDED |    | Check both   |---------+
                +----------+    | confirmed?   |
                                +--------------+
                                       |
                                       | [both confirmed]
                                       v
                                +----------+
                                | COMPLETE |
                                +----------+
    '''
    
    print(state_diagram)
    
    print("\nSTATE TRANSITIONS:")
    transitions = [
        ("AWAITING_PAYMENT", "AWAITING_DELIVERY", "deposit()", "Buyer"),
        ("AWAITING_DELIVERY", "AWAITING_DELIVERY", "confirmDelivery()", "Buyer or Seller (first)"),
        ("AWAITING_DELIVERY", "COMPLETE", "confirmDelivery()", "Buyer or Seller (second)"),
        ("AWAITING_DELIVERY", "REFUNDED", "refund()", "Arbiter only"),
    ]
    
    print(f"\n{'From State':<20} {'To State':<20} {'Function':<20} {'Who Can Call'}")
    print("-" * 80)
    for from_s, to_s, func, who in transitions:
        print(f"{from_s:<20} {to_s:<20} {func:<20} {who}")
    
    print("\n" + "=" * 70)

visualize_escrow_state_machine()

In [ ]:
def visualize_token_state_changes():
    """
    Visualize how token balances change with each operation.
    """
    print("\n" + "=" * 70)
    print("TOKEN CONTRACT STATE CHANGES")
    print("=" * 70)
    
    # Simulated state changes
    print("\nScenario: Alice deploys token, transfers to Bob, Bob approves Exchange")
    print("\nSTEP 1: Deploy (1,000,000 initial supply)")
    print("-" * 50)
    print(f"{'Address':<15} {'Balance':>20}")
    print("-" * 50)
    print(f"{'Alice':<15} {'1,000,000.0000':>20}")
    print(f"{'Bob':<15} {'0.0000':>20}")
    print(f"{'Exchange':<15} {'0.0000':>20}")
    
    print("\nSTEP 2: Alice transfers 10,000 to Bob")
    print("-" * 50)
    print(f"{'Address':<15} {'Balance':>20} {'Change':>15}")
    print("-" * 50)
    print(f"{'Alice':<15} {'990,000.0000':>20} {'-10,000':>15}")
    print(f"{'Bob':<15} {'10,000.0000':>20} {'+10,000':>15}")
    print(f"{'Exchange':<15} {'0.0000':>20} {'--':>15}")
    
    print("\nSTEP 3: Bob approves Exchange for 5,000")
    print("-" * 50)
    print(f"{'Allowance':<25} {'Amount':>20}")
    print("-" * 50)
    print(f"{'Bob -> Exchange':<25} {'5,000.0000':>20}")
    
    print("\nSTEP 4: Exchange calls transferFrom(Bob, Charlie, 3,000)")
    print("-" * 50)
    print(f"{'Address':<15} {'Balance':>20} {'Change':>15}")
    print("-" * 50)
    print(f"{'Bob':<15} {'7,000.0000':>20} {'-3,000':>15}")
    print(f"{'Charlie':<15} {'3,000.0000':>20} {'+3,000':>15}")
    print(f"\n{'Allowance':<25} {'Remaining':>20}")
    print("-" * 50)
    print(f"{'Bob -> Exchange':<25} {'2,000.0000':>20}")
    
    print("\n" + "=" * 70)
    print("KEY INSIGHT: Every operation is an atomic state transition.")
    print("If any require() fails, ALL changes revert (nothing is partial).")
    print("=" * 70)

visualize_token_state_changes()

## Section 7: Understanding Gas Costs (Simulated)

In Ethereum, every operation costs **gas**. Gas is a unit measuring computational effort:

- Simple operations (add, subtract): ~3 gas
- Storage writes: 20,000 gas (expensive!)
- Storage reads: 200 gas
- Calling another contract: 700+ gas

Let's simulate gas costs for our contracts:

In [ ]:
# Gas cost constants (approximate, based on Ethereum Yellow Paper)
GAS_COSTS = {
    # Basic operations
    "base_transaction": 21000,        # Minimum cost for any transaction
    "call_data_zero": 4,              # Per zero byte in calldata
    "call_data_nonzero": 16,          # Per non-zero byte in calldata
    
    # Storage operations
    "sstore_new": 20000,              # Writing new value to storage
    "sstore_update": 5000,            # Updating existing value
    "sstore_delete": -15000,          # Refund for clearing storage
    "sload": 200,                     # Reading from storage
    
    # Memory operations
    "memory_expansion": 3,            # Per word of memory expansion
    
    # Computation
    "add": 3,
    "mul": 5,
    "comparison": 3,
    "keccak256_base": 30,
    "keccak256_per_word": 6,
    
    # Logging (events)
    "log_base": 375,
    "log_per_topic": 375,
    "log_per_byte": 8,
    
    # Contract operations
    "create_contract": 32000,
    "call_contract": 700,
}


def estimate_transfer_gas() -> dict:
    """
    Estimate gas cost for an ERC20 transfer.
    """
    breakdown = {
        "base_transaction": GAS_COSTS["base_transaction"],
        "read_sender_balance (sload)": GAS_COSTS["sload"],
        "read_recipient_balance (sload)": GAS_COSTS["sload"],
        "update_sender_balance (sstore)": GAS_COSTS["sstore_update"],
        "update_recipient_balance (sstore)": GAS_COSTS["sstore_update"],
        "emit_transfer_event": GAS_COSTS["log_base"] + 3 * GAS_COSTS["log_per_topic"] + 32 * GAS_COSTS["log_per_byte"],
        "require_checks": GAS_COSTS["comparison"] * 3,
        "arithmetic": GAS_COSTS["add"] * 2,
    }
    breakdown["TOTAL"] = sum(breakdown.values())
    return breakdown


def estimate_approve_gas() -> dict:
    """
    Estimate gas cost for ERC20 approve.
    """
    breakdown = {
        "base_transaction": GAS_COSTS["base_transaction"],
        "write_allowance (sstore)": GAS_COSTS["sstore_new"],  # Could be new mapping entry
        "emit_approval_event": GAS_COSTS["log_base"] + 3 * GAS_COSTS["log_per_topic"] + 32 * GAS_COSTS["log_per_byte"],
    }
    breakdown["TOTAL"] = sum(breakdown.values())
    return breakdown


def estimate_escrow_deposit_gas() -> dict:
    """
    Estimate gas cost for escrow deposit.
    """
    breakdown = {
        "base_transaction": GAS_COSTS["base_transaction"],
        "read_state (sload)": GAS_COSTS["sload"],
        "read_buyer (sload)": GAS_COSTS["sload"],
        "write_amount (sstore)": GAS_COSTS["sstore_new"],
        "write_state (sstore)": GAS_COSTS["sstore_update"],
        "emit_deposit_event": GAS_COSTS["log_base"] + 2 * GAS_COSTS["log_per_topic"] + 64 * GAS_COSTS["log_per_byte"],
        "require_checks": GAS_COSTS["comparison"] * 3,
    }
    breakdown["TOTAL"] = sum(breakdown.values())
    return breakdown


print("=" * 70)
print("GAS COST ESTIMATION")
print("=" * 70)

# ERC20 Transfer
print("\n1. ERC20 TOKEN TRANSFER")
print("-" * 50)
transfer_gas = estimate_transfer_gas()
for operation, cost in transfer_gas.items():
    if operation == "TOTAL":
        print("-" * 50)
    print(f"  {operation:<40} {cost:>8,} gas")

# ERC20 Approve
print("\n2. ERC20 APPROVE")
print("-" * 50)
approve_gas = estimate_approve_gas()
for operation, cost in approve_gas.items():
    if operation == "TOTAL":
        print("-" * 50)
    print(f"  {operation:<40} {cost:>8,} gas")

# Escrow Deposit
print("\n3. ESCROW DEPOSIT")
print("-" * 50)
deposit_gas = estimate_escrow_deposit_gas()
for operation, cost in deposit_gas.items():
    if operation == "TOTAL":
        print("-" * 50)
    print(f"  {operation:<40} {cost:>8,} gas")

print("\n" + "=" * 70)

In [ ]:
def calculate_transaction_cost(gas_used: int, gas_price_gwei: float = 25.0) -> dict:
    """
    Calculate actual transaction cost in ETH and USD.
    
    Args:
        gas_used: Amount of gas consumed
        gas_price_gwei: Gas price in Gwei (default 25)
    """
    gas_price_wei = gas_price_gwei * 10**9
    cost_wei = gas_used * gas_price_wei
    cost_eth = cost_wei / 10**18
    cost_usd = cost_eth * 2000  # Assume $2000/ETH
    
    return {
        "gas_used": gas_used,
        "gas_price_gwei": gas_price_gwei,
        "cost_eth": cost_eth,
        "cost_usd": cost_usd
    }


print("\n" + "=" * 70)
print("TRANSACTION COST COMPARISON")
print("=" * 70)

operations = [
    ("Simple ETH Transfer", 21000),
    ("ERC20 Transfer", transfer_gas["TOTAL"]),
    ("ERC20 Approve", approve_gas["TOTAL"]),
    ("Escrow Deposit", deposit_gas["TOTAL"]),
    ("Uniswap Swap (typical)", 150000),
    ("NFT Mint (typical)", 85000),
    ("Deploy Token Contract", 500000),
]

print(f"\nAt Gas Price: 25 Gwei, ETH Price: $2,000\n")
print(f"{'Operation':<30} {'Gas':>12} {'Cost (ETH)':>15} {'Cost (USD)':>12}")
print("-" * 75)

for op_name, gas in operations:
    cost = calculate_transaction_cost(gas)
    print(f"{op_name:<30} {gas:>12,} {cost['cost_eth']:>15.6f} ${cost['cost_usd']:>11.2f}")

print("\n" + "-" * 75)
print("\nGAS PRICE SENSITIVITY (for ERC20 Transfer):")
print("-" * 50)

gas_prices = [10, 25, 50, 100, 200]
transfer_gas_amount = transfer_gas["TOTAL"]

print(f"{'Gas Price (Gwei)':<20} {'Cost (USD)':>15}")
print("-" * 35)
for gp in gas_prices:
    cost = calculate_transaction_cost(transfer_gas_amount, gp)
    print(f"{gp:<20} ${cost['cost_usd']:>14.2f}")

print("\n" + "=" * 70)

## Section 8: Common Vulnerabilities (Educational)

Smart contracts handle real money and are immutable. Bugs can be catastrophic. Let's examine common vulnerabilities:

### 8.1 Reentrancy Attack

The most famous vulnerability - it caused the DAO hack ($60M loss in 2016).

In [ ]:
# Vulnerable contract example (DO NOT USE IN PRODUCTION)

VULNERABLE_WITHDRAW = '''
// VULNERABLE CODE - DO NOT USE!
contract VulnerableBank {
    mapping(address => uint256) public balances;
    
    function deposit() external payable {
        balances[msg.sender] += msg.value;
    }
    
    // BUG: Updates state AFTER sending ETH
    function withdraw() external {
        uint256 balance = balances[msg.sender];
        require(balance > 0, "No balance");
        
        // VULNERABLE: External call before state update
        (bool success, ) = msg.sender.call{value: balance}("");
        require(success, "Transfer failed");
        
        // State update happens AFTER external call
        balances[msg.sender] = 0;  // TOO LATE!
    }
}

// Attacker contract
contract Attacker {
    VulnerableBank public bank;
    
    constructor(address _bank) {
        bank = VulnerableBank(_bank);
    }
    
    function attack() external payable {
        bank.deposit{value: 1 ether}();
        bank.withdraw();
    }
    
    // This function is called when receiving ETH
    receive() external payable {
        // Re-enter the withdraw function!
        if (address(bank).balance >= 1 ether) {
            bank.withdraw();  // Recursive call
        }
    }
}
'''

print("REENTRANCY VULNERABILITY")
print("=" * 70)
print(VULNERABLE_WITHDRAW)
print("=" * 70)

In [ ]:
# Simulate the reentrancy attack

class VulnerableBank:
    """Simulated vulnerable contract."""
    
    def __init__(self):
        self.balances: Dict[str, int] = {}
        self.contract_balance = 0
        self.call_log: List[str] = []
    
    def deposit(self, sender: str, amount: int):
        self.balances[sender] = self.balances.get(sender, 0) + amount
        self.contract_balance += amount
        self.call_log.append(f"deposit({sender[:8]}..., {amount})")
    
    def withdraw_vulnerable(self, sender: str, callback: Callable = None):
        """VULNERABLE: State update after external call."""
        balance = self.balances.get(sender, 0)
        if balance <= 0:
            return
        
        self.call_log.append(f"withdraw({sender[:8]}...) - balance: {balance}")
        
        # Send ETH (external call) - this is where reentrancy happens
        self.contract_balance -= balance
        self.call_log.append(f"  -> Sending {balance} ETH to {sender[:8]}...")
        
        # Callback simulates attacker's receive() function
        if callback and self.contract_balance >= balance:
            callback()  # REENTRANCY!
        
        # State update happens AFTER callback (TOO LATE!)
        self.balances[sender] = 0
        self.call_log.append(f"  -> Set balance to 0")


# Simulate attack
print("=" * 70)
print("REENTRANCY ATTACK SIMULATION")
print("=" * 70)

bank = VulnerableBank()

# Setup: Bank has 10 ETH from various users
bank.deposit("0xUser1...", 3)
bank.deposit("0xUser2...", 4)
bank.deposit("0xUser3...", 3)

print(f"\nInitial bank balance: {bank.contract_balance} ETH")

# Attacker deposits 1 ETH
ATTACKER = "0xAttacker"
bank.deposit(ATTACKER, 1)
print(f"Attacker deposits 1 ETH")
print(f"Bank balance: {bank.contract_balance} ETH")

# Attack!
print("\n--- ATTACK BEGINS ---")
stolen = 0

def attacker_callback():
    global stolen
    stolen += 1
    if bank.contract_balance >= 1:
        bank.withdraw_vulnerable(ATTACKER, attacker_callback)

bank.withdraw_vulnerable(ATTACKER, attacker_callback)
stolen += 1  # Count the initial withdrawal

print("\n--- ATTACK COMPLETE ---")
print(f"\nCall sequence:")
for log in bank.call_log:
    print(f"  {log}")

print(f"\nBank balance: {bank.contract_balance} ETH (should have 10 ETH left)")
print(f"Attacker stole: {stolen} ETH (deposited only 1 ETH!)")

print("\n" + "=" * 70)

In [ ]:
# The fix: Checks-Effects-Interactions pattern

SAFE_WITHDRAW = '''
// SAFE CODE - Checks-Effects-Interactions Pattern
contract SafeBank {
    mapping(address => uint256) public balances;
    
    function withdraw() external {
        // CHECKS
        uint256 balance = balances[msg.sender];
        require(balance > 0, "No balance");
        
        // EFFECTS (update state BEFORE external call)
        balances[msg.sender] = 0;
        
        // INTERACTIONS (external call LAST)
        (bool success, ) = msg.sender.call{value: balance}("");
        require(success, "Transfer failed");
    }
}

// Alternative: Use ReentrancyGuard from OpenZeppelin
import "@openzeppelin/contracts/security/ReentrancyGuard.sol";

contract SafeBankWithGuard is ReentrancyGuard {
    mapping(address => uint256) public balances;
    
    // nonReentrant modifier prevents reentrancy
    function withdraw() external nonReentrant {
        uint256 balance = balances[msg.sender];
        require(balance > 0, "No balance");
        
        balances[msg.sender] = 0;
        
        (bool success, ) = msg.sender.call{value: balance}("");
        require(success, "Transfer failed");
    }
}
'''

print("THE FIX: Checks-Effects-Interactions Pattern")
print("=" * 70)
print(SAFE_WITHDRAW)
print("=" * 70)

In [ ]:
# Integer Overflow/Underflow (historical - fixed in Solidity 0.8+)

print("\n" + "=" * 70)
print("INTEGER OVERFLOW/UNDERFLOW")
print("=" * 70)

print("\nBefore Solidity 0.8.0:")
print("-" * 50)

# Simulate uint256 overflow (before 0.8.0)
MAX_UINT256 = 2**256 - 1

print(f"\nMAX_UINT256 = {MAX_UINT256}")
print(f"           = 0x{'f' * 64}")

# Overflow example
value = MAX_UINT256
overflow_result = (value + 1) % (2**256)  # Simulate overflow
print(f"\nOverflow: MAX_UINT256 + 1 = {overflow_result}")
print("  (wraps around to 0!)")

# Underflow example
value = 0
underflow_result = (value - 1) % (2**256)  # Simulate underflow
print(f"\nUnderflow: 0 - 1 = {underflow_result}")
print("  (wraps around to MAX_UINT256!)")

# Attack scenario
print("\n" + "-" * 50)
print("ATTACK SCENARIO:")
print("-" * 50)
print('''
// Vulnerable token contract
function transfer(address to, uint256 amount) public {
    // BUG: No underflow check before Solidity 0.8
    balances[msg.sender] -= amount;  // Could underflow!
    balances[to] += amount;
}

// Attacker has 0 tokens, transfers 1:
// balances[attacker] = 0 - 1 = MAX_UINT256
// Now attacker has unlimited tokens!
''')

print("\n" + "=" * 70)
print("SOLUTION: Solidity 0.8+ has built-in overflow checks")
print("=" * 70)
print('''
// Solidity 0.8+ automatically reverts on overflow/underflow
function transfer(address to, uint256 amount) public {
    balances[msg.sender] -= amount;  // Reverts if would underflow
    balances[to] += amount;          // Reverts if would overflow
}

// For intentional wrapping, use unchecked:
unchecked {
    // Gas optimization when overflow is intentional/impossible
}
''')

In [ ]:
# Summary of common vulnerabilities

print("\n" + "=" * 70)
print("SMART CONTRACT VULNERABILITY SUMMARY")
print("=" * 70)

vulnerabilities = [
    ("Reentrancy", 
     "External calls before state updates",
     "Checks-Effects-Interactions, ReentrancyGuard",
     "$60M (The DAO, 2016)"),
    
    ("Integer Overflow/Underflow",
     "Arithmetic wrapping in older Solidity",
     "Use Solidity 0.8+, SafeMath library",
     "$40M+ (Various ERC20 tokens)"),
    
    ("Access Control",
     "Missing or incorrect permission checks",
     "Proper modifiers, OpenZeppelin AccessControl",
     "$30M+ (Parity Wallet, 2017)"),
    
    ("Front-running",
     "MEV bots exploit pending transactions",
     "Commit-reveal schemes, Flashbots Protect",
     "Ongoing (millions daily)"),
    
    ("Oracle Manipulation",
     "Manipulating price feeds for profit",
     "TWAP oracles, Chainlink, multiple sources",
     "$100M+ (Flash loan attacks)"),
    
    ("Denial of Service",
     "Blocking contract functionality",
     "Pull over push, gas limits, timeouts",
     "$150M (King of Ether)"),
]

for vuln_name, description, prevention, damage in vulnerabilities:
    print(f"\n{vuln_name}")
    print(f"  Description: {description}")
    print(f"  Prevention:  {prevention}")
    print(f"  Notable Loss: {damage}")

print("\n" + "=" * 70)
print("KEY TAKEAWAY: Smart contract security is critical!")
print("Always:")
print("  1. Use audited libraries (OpenZeppelin)")
print("  2. Get professional audits")
print("  3. Test extensively (fuzzing, formal verification)")
print("  4. Use bug bounties")
print("  5. Start with small amounts / timelock upgrades")
print("=" * 70)

## Section 9: Challenge Exercises

### Challenge 1: Implement a Voting Contract

Create a simple voting contract with the following features:
- Owner can create proposals
- Token holders can vote (one token = one vote)
- Proposals have a deadline
- After deadline, anyone can execute if majority voted yes

In [ ]:
# Challenge 1: Implement a Voting Contract

class VotingContract:
    """
    TODO: Implement a simple voting contract.
    
    Requirements:
    - create_proposal(caller, description, deadline): Create new proposal
    - vote(caller, proposal_id, support): Vote yes/no on proposal
    - execute_proposal(caller, proposal_id): Execute if passed
    - get_proposal_status(proposal_id): View proposal details
    
    State:
    - proposals: Dict[int, Proposal]
    - votes: Dict[int, Dict[str, bool]]  # proposal_id -> voter -> support
    - token_balances: Dict[str, int]  # voter -> balance (voting power)
    """
    
    def __init__(self, owner: str, token_contract: SimulatedTokenContract):
        self.owner = owner
        self.token = token_contract
        self.proposals = {}
        self.votes = {}
        self.next_proposal_id = 1
        # TODO: Add more initialization
    
    def create_proposal(self, caller: str, description: str, deadline: float) -> int:
        """Create a new proposal. Only owner can create."""
        # TODO: Implement
        pass
    
    def vote(self, caller: str, proposal_id: int, support: bool) -> bool:
        """Vote on a proposal. Voting power = token balance."""
        # TODO: Implement
        pass
    
    def execute_proposal(self, caller: str, proposal_id: int) -> bool:
        """Execute a proposal if it passed and deadline reached."""
        # TODO: Implement
        pass
    
    def get_proposal_status(self, proposal_id: int) -> dict:
        """Get current status of a proposal."""
        # TODO: Implement
        pass


print("Challenge 1: Implement the VotingContract class above!")
print("\nHints:")
print("- Use token.get_balance(address) to get voting power")
print("- Track yes_votes and no_votes separately")
print("- Use time.time() to compare with deadline")
print("- A proposal passes if yes_votes > no_votes")

### Challenge 2: Add Fee Mechanism to Token Contract

Modify the token contract to charge a 0.5% transfer fee that goes to the contract owner.

In [ ]:
# Challenge 2: Token with Transfer Fee

class TokenWithFee(SimulatedTokenContract):
    """
    Token contract with 0.5% transfer fee.
    
    TODO: Override the transfer method to:
    1. Calculate 0.5% fee
    2. Send fee to owner
    3. Send remaining amount to recipient
    """
    
    def __init__(self, deployer: str, name: str, symbol: str, initial_supply: int):
        super().__init__(deployer, name, symbol, initial_supply)
        self.fee_percent = 0.5  # 0.5% fee
        self.total_fees_collected = 0
    
    def transfer(self, sender: str, to: str, amount: int) -> bool:
        """
        Transfer tokens with fee.
        
        TODO: Implement fee logic
        - Calculate fee: amount * 0.005
        - Deduct full amount from sender
        - Send (amount - fee) to recipient
        - Send fee to owner
        - Update total_fees_collected
        """
        # TODO: Implement
        pass


print("Challenge 2: Implement the TokenWithFee class above!")
print("\nTest case:")
print("- Alice sends 1000 tokens to Bob")
print("- Fee: 1000 * 0.5% = 5 tokens")
print("- Bob receives: 995 tokens")
print("- Owner receives: 5 tokens")

### Challenge 3: Implement Time-Locked Wallet

Create a wallet that locks funds until a specific timestamp.

In [ ]:
# Challenge 3: Time-Locked Wallet

class TimeLockWallet:
    """
    A wallet that locks funds until a specific time.
    
    Use cases:
    - Vesting schedules for employees
    - Trust funds that unlock at age 18
    - Delayed gratification savings
    
    TODO: Implement:
    - deposit(caller, amount): Add funds to the lock
    - withdraw(caller): Withdraw ALL funds (only after unlock time)
    - extend_lock(caller, new_unlock_time): Extend the lock (can't shorten)
    - get_status(): View current status
    """
    
    def __init__(self, beneficiary: str, unlock_time: float):
        """
        Create a new time-locked wallet.
        
        Args:
            beneficiary: Address that can withdraw after unlock
            unlock_time: Unix timestamp when funds unlock
        """
        self.beneficiary = beneficiary
        self.unlock_time = unlock_time
        self.balance = 0
        self.is_withdrawn = False
        # TODO: Add more state as needed
    
    def deposit(self, caller: str, amount: int) -> bool:
        """Anyone can deposit funds."""
        # TODO: Implement
        pass
    
    def withdraw(self, caller: str) -> int:
        """Only beneficiary can withdraw, only after unlock time."""
        # TODO: Implement
        pass
    
    def extend_lock(self, caller: str, new_unlock_time: float) -> bool:
        """Only beneficiary can extend lock (not shorten)."""
        # TODO: Implement
        pass
    
    def get_status(self) -> dict:
        """Get current wallet status."""
        # TODO: Implement
        pass


print("Challenge 3: Implement the TimeLockWallet class above!")
print("\nTest scenario:")
print("1. Create wallet unlocking in 30 days")
print("2. Deposit 10 ETH")
print("3. Try to withdraw (should fail - too early)")
print("4. Simulate time passing")
print("5. Withdraw (should succeed)")

## Summary

In this notebook, you learned:

### Key Concepts

1. **What Smart Contracts Are**:
   - Self-executing code stored on a blockchain
   - Automatic enforcement without intermediaries
   - Deterministic, immutable, and transparent

2. **Smart Contract Anatomy (Solidity)**:
   - State variables (permanent storage)
   - Events (logs for notifications)
   - Constructor (runs once at deployment)
   - Modifiers (reusable access control)
   - Functions (callable code)
   - require() for validation

3. **Token Contract (ERC20-like)**:
   - Balances stored in mappings
   - transfer() for direct sends
   - approve() + transferFrom() for DeFi integration
   - Events for tracking transfers

4. **Escrow Contract**:
   - State machine pattern (AWAITING_PAYMENT -> AWAITING_DELIVERY -> COMPLETE)
   - Multi-party confirmation
   - Arbiter for dispute resolution

5. **Gas Economics**:
   - Every operation has a gas cost
   - Storage is expensive (20,000 gas to write)
   - Optimizing gas is crucial for usability

6. **Common Vulnerabilities**:
   - Reentrancy (external calls before state updates)
   - Integer overflow/underflow (fixed in Solidity 0.8+)
   - Access control issues
   - Prevention: Checks-Effects-Interactions, OpenZeppelin, audits

### Why Smart Contracts Matter for Finance

- **DeFi**: Lending, borrowing, trading without banks
- **Tokenization**: Any asset can become a token
- **DAOs**: Democratic organizations governed by code
- **NFTs**: Provable digital ownership
- **Insurance**: Automatic claim payouts

### Next Steps

- NB09: AMM Simulation (Automated Market Makers)
- NB10: Stablecoin Analysis
- NB11: DeFi Exploits Case Studies
- NB12: DAO Governance Mechanisms

### Further Reading

- [Solidity Documentation](https://docs.soliditylang.org/)
- [OpenZeppelin Contracts](https://docs.openzeppelin.com/contracts/)
- [Ethereum Smart Contract Best Practices](https://consensys.github.io/smart-contract-best-practices/)
- [SWC Registry](https://swcregistry.io/) - Smart Contract Weakness Classification
- [Damn Vulnerable DeFi](https://www.damnvulnerabledefi.xyz/) - Security challenges